In [ ]:
import pandas as pd
from coinbase.wallet.client import Client
from sec import api_key, api_sec

client = Client(api_key, api_sec)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [ ]:
def get_paginated_accounts (limit):
    #annab valja listi koikide accountidega
    all_items = []
    starting_after = None
    while True:
        items = client.get_accounts(limit = limit, starting_after = starting_after)
        if items.pagination.next_starting_after is not None:
            starting_after = items.pagination.next_starting_after
            all_items += items.data
        else:
            all_items += items.data
            break
    return all_items

In [ ]:
def get_transaction_table_sorted(accounts):
    all_transactions = pd.DataFrame()

    for account in accounts:
        if len(account['id']) < 10:
            continue
        print('checking ', account.balance.currency)
        local_transactions = pd.DataFrame()
        starting_after = None
        while True:
            transactions = client.get_transactions(account['id'], limit = 100, starting_after = starting_after)
            if transactions.pagination.next_starting_after is not None:
                starting_after = transactions.pagination.next_starting_after
                local_transactions = pd.concat([local_transactions, pd.DataFrame(transactions.data)])
            else:
                local_transactions = pd.concat([local_transactions, pd.DataFrame(transactions.data)])
                break
        if local_transactions.empty:
            continue

        
        local_transactions = local_transactions.sort_values('created_at')
        all_transactions = pd.concat([all_transactions, local_transactions])
    all_transactions.loc[:,'details'] = all_transactions.loc[:,'details'].apply(lambda x: x.get('header'))
    all_transactions.loc[:,'currency'] = all_transactions.loc[:,'amount'].apply(lambda x: x.get('currency'))
    all_transactions.loc[:,'crypto_amount'] = all_transactions.loc[:,'amount'].apply(lambda x: x.get('amount'))
    all_transactions.loc[:,'native_currency'] = all_transactions.loc[:,'native_amount'].apply(lambda x: x.get('currency'))
    all_transactions.loc[:,'native_amount'] = all_transactions.loc[:,'native_amount'].apply(lambda x: x.get('amount'))
    all_transactions['native_amount'] = all_transactions['native_amount'].apply(lambda x: float(x))
    all_transactions['crypto_amount'] = all_transactions['crypto_amount'].apply(lambda x: float(x))
    all_transactions = all_transactions[['id','native_amount','created_at','type','details','currency','crypto_amount','native_currency']]
    return all_transactions

In [ ]:
def get_fifo_table(tabel_of_a_crypto):    
    tabel_of_sales_of_a_buy = pd.DataFrame(columns=['buy_id', 'currency', 'buy_amount_to_this_sell', 'buy_amount_in_native', 'sale_amount_in_crypto', 'sale_amount_in_native', 'buy_ppc', 'sell_ppc', 'profit in %', 'tax_owned_for_this_transaction', 'index_of_sale'])
    tabel_of_a_crypto.loc[:,'to_be_spent_from_this_buy'] = tabel_of_a_crypto['crypto_amount'][tabel_of_a_crypto['crypto_amount'] > 0]
    sales_locations = tabel_of_a_crypto[tabel_of_a_crypto['crypto_amount'] < 0].index
    for sale_index in sales_locations:
        sale = tabel_of_a_crypto.loc[sale_index]
        sale_size = abs(sale.crypto_amount)
        while sale_size > sum(tabel_of_sales_of_a_buy[tabel_of_sales_of_a_buy['index_of_sale'] == sale_index]['buy_amount_to_this_sell']):
            already_spent_on_this_sell = sum(tabel_of_sales_of_a_buy[tabel_of_sales_of_a_buy['index_of_sale'] == sale_index]['buy_amount_to_this_sell'])
            try:
                use_a_new_buy = tabel_of_a_crypto[tabel_of_a_crypto['to_be_spent_from_this_buy'] > 0].iloc[0]
            except:
                break
            buy_ppc = use_a_new_buy['native_amount'] / use_a_new_buy['crypto_amount']
            sell_ppc = sale['native_amount'] / sale['crypto_amount']
            if (use_a_new_buy['to_be_spent_from_this_buy'] + already_spent_on_this_sell) > sale_size:
                spend_from_this_buy = sale_size - already_spent_on_this_sell
                index = tabel_of_a_crypto[tabel_of_a_crypto['id'] == use_a_new_buy['id']].index
                tabel_of_a_crypto.loc[index, 'to_be_spent_from_this_buy'] = use_a_new_buy['to_be_spent_from_this_buy'] - spend_from_this_buy
                temp = (use_a_new_buy['id'], use_a_new_buy['currency'], spend_from_this_buy, use_a_new_buy['native_amount'], sale_size, sale.native_amount, buy_ppc, sell_ppc, 0, 0, sale_index)
                tabel_of_sales_of_a_buy.loc[len(tabel_of_sales_of_a_buy)] = temp
                break
            else:
                temp = (use_a_new_buy['id'], use_a_new_buy['currency'], use_a_new_buy['to_be_spent_from_this_buy'], use_a_new_buy['native_amount'], sale_size, sale.native_amount, buy_ppc, sell_ppc, 0, 0, sale_index)
                index = tabel_of_a_crypto[tabel_of_a_crypto['id'] == use_a_new_buy['id']].index
                tabel_of_a_crypto.loc[index, 'to_be_spent_from_this_buy'] = 0
                tabel_of_sales_of_a_buy.loc[len(tabel_of_sales_of_a_buy)] = temp
    tabel_of_sales_of_a_buy.loc[:, 'profit in %'] = (tabel_of_sales_of_a_buy['sell_ppc'] / tabel_of_sales_of_a_buy['buy_ppc'])*100
    tabel_of_sales_of_a_buy.loc[:, 'tax_owned_for_this_transaction'] = round(( ((tabel_of_sales_of_a_buy['buy_amount_to_this_sell'] / tabel_of_sales_of_a_buy['sale_amount_in_crypto']) * abs(tabel_of_sales_of_a_buy['sale_amount_in_native'])) - (tabel_of_sales_of_a_buy['buy_amount_to_this_sell'] * tabel_of_sales_of_a_buy['buy_ppc']) ) * 0.2, 4)
    tabel_of_sales_of_a_buy['tax_owned_for_this_transaction'][tabel_of_sales_of_a_buy['profit in %'] < 100] = 0  
    return tabel_of_sales_of_a_buy

In [ ]:
def get_fifo_of_all_cryptos(all_transactions):
    total_sold = 0
    currencies = all_transactions.currency.unique()
    the_master_taxation_table = pd.DataFrame()
    for currency in currencies:   
        tabel_of_a_crypto = all_transactions[all_transactions['currency'] == currency]
        
        
        tabel_of_sales_of_a_buy = get_fifo_table(tabel_of_a_crypto)
        the_master_taxation_table = pd.concat([the_master_taxation_table, tabel_of_sales_of_a_buy])
        
        
        unique_sells = tabel_of_sales_of_a_buy.drop_duplicates('index_of_sale')
        total_sold = total_sold + abs(sum(unique_sells['sale_amount_in_native']))

    total_bought = sum(the_master_taxation_table['buy_amount_to_this_sell']*the_master_taxation_table['buy_ppc'])
    print('soetamismaksumus ', total_bought)
    print('müügi-/turuhind', total_sold)
    return the_master_taxation_table    

In [ ]:
def weighted_average(tabel_of_a_crypto):
    sales_locations = []
    sales_locations = tabel_of_a_crypto[tabel_of_a_crypto['native_amount'] < 0].index
    for sale in sales_locations:
        buys_before_sell_temp = tabel_of_a_crypto.iloc[:len(tabel_of_a_crypto) - sale]
        buys_before_sell = buys_before_sell_temp[buys_before_sell_temp['native_amount'] > 0]
        average = sum(buys_before_sell['native_amount']) / sum(buys_before_sell['crypto_amount'])
        tabel_of_a_crypto.loc[sale,'average_buy_price_before_this_sell'] = average
        tabel_of_a_crypto.loc[sale, 'initial_investement_to_this_sell'] = abs(average * tabel_of_a_crypto.loc[sale, 'crypto_amount'])
        tabel_of_a_crypto.loc[sale, 'profit'] = abs(tabel_of_a_crypto.loc[sale, 'native_amount']) - tabel_of_a_crypto.loc[sale, 'initial_investement_to_this_sell']
        tabel_of_a_crypto.loc[sale, 'tax_owned_for_this_txs'] = tabel_of_a_crypto.loc[sale, 'profit'] * 0.2
    return tabel_of_a_crypto
    

In [ ]:
def get_weighed_average_of_all(all_transactions):
    currencies = all_transactions.currency.unique()
    the_master_taxation_table = pd.DataFrame()

    for currency in currencies:
        tabel_of_a_crypto = all_transactions[all_transactions['currency'] == currency]
        weighted_average_of_a_crypto = weighted_average(tabel_of_a_crypto)
        the_master_taxation_table = pd.concat([the_master_taxation_table, weighted_average_of_a_crypto])
    the_master_taxation_table['tax_owned_for_this_txs'][the_master_taxation_table['tax_owned_for_this_txs'] < 0]
    sells = the_master_taxation_table[the_master_taxation_table['crypto_amount'] < 0]
    total_bought = sum(sells['crypto_amount'] * sells['average_buy_price_before_this_sell'])
    total_sold = sum(sells['native_amount'])
    print('soetamismaksumus ', abs(total_bought))
    print('müügi-/turuhind', abs(total_sold))
    return the_master_taxation_table



In [ ]:
accounts = get_paginated_accounts(limit = 100)
all_transactions = get_transaction_table_sorted(accounts) #gets all transactions

In [ ]:
the_master_table = get_weighed_average_of_all(all_transactions) #gets weighed average of all transactions

In [ ]:
the_master_table = get_fifo_of_all_cryptos(all_transactions) #gets fifo of all transactions

In [ ]:
the_master_table